<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.sigmoid(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 80:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.09375
Current benign train loss: 2.321267604827881

Current batch: 100
Current benign train accuracy: 0.1953125
Current benign train loss: 8.89016056060791

Current batch: 200
Current benign train accuracy: 0.1953125
Current benign train loss: 2.1560466289520264

Current batch: 300
Current benign train accuracy: 0.203125
Current benign train loss: 2.0616791248321533

Total benign train accuarcy: 17.74
Total benign train loss: 1432.148018360138

[ Test epoch: 0 ]

Test accuarcy: 23.04
Test average loss: 0.021706027209758758
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.2734375
Current benign train loss: 2.1345036029815674

Current batch: 100
Current benign train accuracy: 0.2890625
Current benign train loss: 1.9635039567947388

Current batch: 200
Current benign train accuracy: 0.265625
Current benign train loss: 2.1573123931884766

Current batch: 300
Current benign train accuracy


Current batch: 200
Current benign train accuracy: 0.734375
Current benign train loss: 0.7334268093109131

Current batch: 300
Current benign train accuracy: 0.703125
Current benign train loss: 0.8061923980712891

Total benign train accuarcy: 64.908
Total benign train loss: 386.4324007034302

[ Test epoch: 13 ]

Test accuarcy: 22.5
Test average loss: 0.0609583634853363
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.609375
Current benign train loss: 1.2825181484222412

Current batch: 100
Current benign train accuracy: 0.75
Current benign train loss: 0.8323313593864441

Current batch: 200
Current benign train accuracy: 0.5625
Current benign train loss: 1.1245242357254028

Current batch: 300
Current benign train accuracy: 0.6953125
Current benign train loss: 0.8582118153572083

Total benign train accuarcy: 66.72
Total benign train loss: 365.9318122267723

[ Test epoch: 14 ]

Test accuarcy: 42.23
Test average loss: 0.02268699526786804
Model Saved!

[ Tr


Total benign train accuarcy: 74.546
Total benign train loss: 288.12025341391563

[ Test epoch: 26 ]

Test accuarcy: 29.64
Test average loss: 0.0440602498292923
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.6953125
Current benign train loss: 0.7982678413391113

Current batch: 100
Current benign train accuracy: 0.6953125
Current benign train loss: 0.9528117179870605

Current batch: 200
Current benign train accuracy: 0.7421875
Current benign train loss: 0.7369554042816162

Current batch: 300
Current benign train accuracy: 0.7109375
Current benign train loss: 0.8374494314193726

Total benign train accuarcy: 74.774
Total benign train loss: 285.08096891641617

[ Test epoch: 27 ]

Test accuarcy: 12.22
Test average loss: 0.0911129506111145
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.7421875
Current benign train loss: 0.7632591724395752

Current batch: 100
Current benign train accuracy: 0.7578125
Current benign tra


Current batch: 0
Current benign train accuracy: 0.7421875
Current benign train loss: 0.8225029706954956

Current batch: 100
Current benign train accuracy: 0.6953125
Current benign train loss: 0.9885900020599365

Current batch: 200
Current benign train accuracy: 0.84375
Current benign train loss: 0.5167645215988159

Current batch: 300
Current benign train accuracy: 0.8046875
Current benign train loss: 0.55889892578125

Total benign train accuarcy: 77.718
Total benign train loss: 255.3244933784008

[ Test epoch: 40 ]

Test accuarcy: 18.44
Test average loss: 0.05814416418075562
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.734375
Current benign train loss: 0.694489061832428

Current batch: 100
Current benign train accuracy: 0.828125
Current benign train loss: 0.5827121138572693

Current batch: 200
Current benign train accuracy: 0.7265625
Current benign train loss: 0.761242687702179

Current batch: 300
Current benign train accuracy: 0.7890625
Current


Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.35101068019866943

Current batch: 300
Current benign train accuracy: 0.8671875
Current benign train loss: 0.3935161232948303

Total benign train accuarcy: 86.822
Total benign train loss: 149.38678266108036

[ Test epoch: 53 ]

Test accuarcy: 82.86
Test average loss: 0.005331038403511047
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2836610674858093

Current batch: 100
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3640846014022827

Current batch: 200
Current benign train accuracy: 0.8828125
Current benign train loss: 0.37146398425102234

Current batch: 300
Current benign train accuracy: 0.84375
Current benign train loss: 0.46552586555480957

Total benign train accuarcy: 87.102
Total benign train loss: 146.79715839028358

[ Test epoch: 54 ]

Test accuarcy: 72.82
Test average loss: 0.009452317559719085


Total benign train accuarcy: 88.298
Total benign train loss: 133.15651048719883

[ Test epoch: 66 ]

Test accuarcy: 72.92
Test average loss: 0.0088609772503376
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 0.921875
Current benign train loss: 0.19453629851341248

Current batch: 100
Current benign train accuracy: 0.8515625
Current benign train loss: 0.3734501898288727

Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2724010944366455

Current batch: 300
Current benign train accuracy: 0.84375
Current benign train loss: 0.4507666826248169

Total benign train accuarcy: 88.044
Total benign train loss: 133.35057039558887

[ Test epoch: 67 ]

Test accuarcy: 72.72
Test average loss: 0.010861999917030334
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 0.9140625
Current benign train loss: 0.37132352590560913

Current batch: 100
Current benign train accuracy: 0.9296875
Current benign tr


Current batch: 0
Current benign train accuracy: 0.890625
Current benign train loss: 0.3005642592906952

Current batch: 100
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3063473105430603

Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.24020081758499146

Current batch: 300
Current benign train accuracy: 0.8984375
Current benign train loss: 0.26428189873695374

Total benign train accuarcy: 90.076
Total benign train loss: 114.42030575871468

[ Test epoch: 80 ]

Test accuarcy: 87.82
Test average loss: 0.0036648579955101015
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 0.8359375
Current benign train loss: 0.4841286838054657

Current batch: 100
Current benign train accuracy: 0.875
Current benign train loss: 0.3352971076965332

Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.26457294821739197

Current batch: 300
Current benign train accuracy: 0.914062


Current batch: 200
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2883909344673157

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.22155483067035675

Total benign train accuarcy: 90.984
Total benign train loss: 103.0636992752552

[ Test epoch: 93 ]

Test accuarcy: 87.84
Test average loss: 0.003631664763391018
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.19133856892585754

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.26505693793296814

Current batch: 200
Current benign train accuracy: 0.921875
Current benign train loss: 0.2452218383550644

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.26028013229370117

Total benign train accuarcy: 91.016
Total benign train loss: 101.97406460344791

[ Test epoch: 94 ]

Test accuarcy: 87.56
Test average loss: 0.0037154083892703056
Mod